In [1]:
import requests
from requests.exceptions import HTTPError, Timeout, RequestException
import json
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import mysql.connector
from sqlalchemy import create_engine
from data import TAXES_LIST, TICKER_LIST


# Getting envoironment variables and defining constants
load_dotenv('token.env')
BRAPI_TOKEN = os.getenv('BRAPI_TOKEN')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD= os.getenv('DB_PASSWORD')
DB_SERVER = os.getenv('DB_SERVER')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
BRAPI_BASE_URL = 'https://brapi.dev/api'

In [12]:
# Exibir todas as colunas
pd.set_option('display.max_columns', None)

# Exibir todas as linhas
pd.set_option('display.max_rows', None)

In [2]:
def get_response_brapi(endpoint, params):
  
  try:
    response = requests.get(f'{BRAPI_BASE_URL}{endpoint}', params=params)
    return response.json()
  
  except HTTPError as http_err:
      print(f"Erro HTTP ocorreu: {http_err}")
  except Timeout as timeout_err:
      print(f"A requisição demorou muito tempo: {timeout_err}")
  except RequestException as req_err:
      print(f"Ocorreu um erro na requisição: {req_err}")
  except Exception as err:
      print(f"Ocorreu um erro inesperado: {err}")
      
  return

In [3]:
def extract_company_data(ticker:str) -> dict:
  
  """
  get all data related to the company from brapi api
  
  Args:
    ticker(str): code of the finance asset
    
  Returns:
    response(dict): data returned from api in json format
  """  

  endpoint = f'/quote/{ticker}'
  params = {
      'token': BRAPI_TOKEN,
      'range': '1d',
      'interval': '1d',
      'fundamental': 'true',
      'modules': ['summaryProfile']
  }

  response = get_response_brapi(endpoint, params)

  return response

In [4]:
def transform_company_data(response:str) -> dict:

  data = response['results']
  
  for result in data:
    summary_profile = result.pop('summaryProfile', {})
    result.update(summary_profile)

    first_historical_record = result['historicalDataPrice'][0]
    result.update(first_historical_record)
    result.pop('historicalDataPrice', {})
    
    result.pop('validRanges', {})
    
    result.pop('validIntervals', {})

  return data

In [ ]:
def load_data(df:pd.DataFrame, table:str, schema:str) -> None:

  connection = f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_SERVER}:{DB_PORT}/{DB_NAME}'
  engine = create_engine(connection)

  df.to_sql(table, con=engine, schema=schema, if_exists='replace', index=False)
  
  return

In [5]:
def get_companies(companies:dict) -> None:
  
  data = []
  table = 'empresas'
  schema = 'pi6'
  
  for ticker, company in companies.items():
    response = extract_company_data(ticker)
    data.append(transform_company_data(response))
    
  df = pd.DataFrame(data)  
  load_data(df, table, schema)
    
  return

In [16]:
def extract_taxes_data(serie_code:str, start_date:str, end_date:str) -> dict:

  format = 'json'

  url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{serie_code}/dados?formato={format}&dataInicial={start_date}&dataFinal={end_date}'

  try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()
  except HTTPError as http_err:
      print(f"Erro HTTP ocorreu: {http_err}")
  except Timeout as timeout_err:
      print(f"Timeout Error: {timeout_err}")
  except RequestException as req_err:
      print(f"Request Error: {req_err}")
  except Exception as err:
      print(f"Unexpected Error: {err}")

  data = response.json()

  return data 

In [17]:
def transform_taxes_data(data:dict, tax_name:str, start_date, end_date):
  
  df = pd.DataFrame(data)
  start_datetime = pd.to_datetime(start_date, dayfirst=True)
  end_datetime = pd.to_datetime(end_date, dayfirst=True)
  
  df.rename(columns={'valor':tax_name}, inplace=True)
  
  df['data'] = pd.to_datetime(df['data'], dayfirst=True)
  all_dates = pd.date_range(start=start_datetime, end=end_datetime)
  df = df.set_index('data').reindex(all_dates).ffill().reset_index()

  return df 

In [18]:
def get_taxes_data(taxes:dict):
  
  table = 'taxas'
  schema = 'pi6'
  
  start_date = '01/01/2023'
  end_date = '31/12/2023'
  
  df = pd.DataFrame()
  
  for tax_name, serie_code in taxes.items():
    data = extract_taxes_data(serie_code, start_date, end_date)
    df1 = transform_taxes_data(data, tax_name, start_date, end_date)
    df = pd.merge(df, df1, on='data', how='outer')
    df = df.sort_values(by='data').reset_index(drop=True)
    
  load_data(df, table, schema)
  
  return

In [21]:
table = 'taxas'
schema = 'pi6'

start_date = '01/01/2023'
end_date = '31/12/2023'

df = pd.DataFrame()

for tax_name, serie_code in TAXES_LIST.items():
  data = extract_taxes_data(serie_code, start_date, end_date)
  df1 = transform_taxes_data(data, tax_name, start_date, end_date)
  print(df1)

         index  SELIC
0   2023-01-01    NaN
1   2023-01-02  13.65
2   2023-01-03  13.65
3   2023-01-04  13.65
4   2023-01-05  13.65
5   2023-01-06  13.65
6   2023-01-07  13.65
7   2023-01-08  13.65
8   2023-01-09  13.65
9   2023-01-10  13.65
10  2023-01-11  13.65
11  2023-01-12  13.65
12  2023-01-13  13.65
13  2023-01-14  13.65
14  2023-01-15  13.65
15  2023-01-16  13.65
16  2023-01-17  13.65
17  2023-01-18  13.65
18  2023-01-19  13.65
19  2023-01-20  13.65
20  2023-01-21  13.65
21  2023-01-22  13.65
22  2023-01-23  13.65
23  2023-01-24  13.65
24  2023-01-25  13.65
25  2023-01-26  13.65
26  2023-01-27  13.65
27  2023-01-28  13.65
28  2023-01-29  13.65
29  2023-01-30  13.65
30  2023-01-31  13.65
31  2023-02-01  13.65
32  2023-02-02  13.65
33  2023-02-03  13.65
34  2023-02-04  13.65
35  2023-02-05  13.65
36  2023-02-06  13.65
37  2023-02-07  13.65
38  2023-02-08  13.65
39  2023-02-09  13.65
40  2023-02-10  13.65
41  2023-02-11  13.65
42  2023-02-12  13.65
43  2023-02-13  13.65
44  2023-0